# Prototype

## Importing packages and setting up API/LOGGING

In [101]:
import os
import logging
import requests
import datetime

import pandas as pd

from dotenv import load_dotenv, set_key

In [2]:
# Setting up the API
authURL = "https://www.warcraftlogs.com/oauth/authorize"
tokenURL= "https://www.warcraftlogs.com/oauth/token"
api_key = os.getenv('client_secret')


In [3]:
# Setting up logging
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s - %(name)s -%(levelname)s -%(message)s",
                    handlers=[logging.StreamHandler()])

logger = logging.getLogger(__name__)

## Functions

In [147]:
#Funktions for adding gameID data
def make_gameID_query(report_code):
    query = f"""query PlayerDungeonMetrics{{
                                            reportData{{
                                                report(code: "{report_code}"){{                        
                                                    masterData{{
                                                        actors(type: "Player"){{
                                                            name
                                                            gameID
                                                            }}
                                                        }}
                                                    }}
                                                }}
                                            }}""" 
    return query

def get_gameID(token, report_code):
    gameID_query = make_query(token, make_gameID_query(report_code))
    logger.info("gameID_query successful.")
    df_gameID = pd.json_normalize(gameID_query, record_path=['data', 'reportData', 'report', 'masterData', 'actors'])
    return df_gameID

In [4]:
# Functions for handling the token needed for authorization. 

def read_token(token_name='WARCRAFTLOGS_TOKEN'):
    """
    Reads a token from a .env file.

    This function first loads environment variables from a .env file
    if it exists. It then attempts to retrieve the specified token
    from the environment.

    Args:
        token_name (str): The name of the environment variable that holds
                          the token.

    Returns:
        str or None: The token string if found, otherwise None.
    """
    load_dotenv()
    token = os.getenv(token_name)

    if token is None:
        print(f"Error: The token '{token_name}' was not found in the .env file.")
        logger.info(f"Error: The token '{token_name}' was not found in the .env file.")
        return None

    return token

def store_token(token, token_name='WARCRAFTLOGS_TOKEN'):
    """
    Saves a new token to the .env file.

    Args:
        token (str): The token to be saved.
        token_name (str): The name of the environment variable.
    """
    # Get the path to the .env file, assuming it's in the same directory.
    dotenv_path = os.path.join(os.getcwd(), '.env')
    set_key(dotenv_path, token_name, token)

    print(f"Successfully saved new token to the .env file under key '{token_name}'.")
    logger.info(f"Successfully saved new token to the .env file under key '{token_name}'.")

def get_new_token(client_id, client_secret):
    """
    Gets a new access token from the Warcraft Logs API using the Client Credentials flow.
    If successful, it saves the new token to the .env file.

    Args:
        client_id (str): The public client ID for your application.
        client_secret (str): The confidential client secret for your application.

    Returns:
        str or None: The new access token string if successful, otherwise None.
    """
    url = "https://www.warcraftlogs.com/oauth/token"
    data = {'grant_type': 'client_credentials'}

    try:
        response = requests.post(url, data=data, auth=(client_id, client_secret))

        token_data = response.json()
        access_token = token_data.get('access_token')

        if access_token:
            print("Successfully retrieved a new access token.")
            logger.info("Successfully retrieved a new access token.")
            store_token(access_token)
            return access_token
        else:
            print("Error: Access token not found in the API response.")
            logger.info("Error: Access token not found in the API response.")
            return None

    except requests.exceptions.RequestException as e:
        print(f"An error occurred while getting a new token: {e}")
        return None

In [146]:
# Base function for making querys
def make_query(token, query):
    """
    Makes a GraphQL query to the Warcraft Logs API using the provided access token.

    Args:
        token (str): The access token to use for authorization.
        query (str): The GraphQL query string.

    Returns:
        dict or None: The JSON response data if successful, otherwise None.
    """
    url = "https://www.warcraftlogs.com/api/v2/client"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    data = {'query': query}

    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while making the GraphQL query: {e}")
        return None


In [11]:
# Functions and querys for damage and healing
def make_damage_query(report_code):
    query = f"""query PlayerDungeonMetrics{{
                                            reportData{{
                                                report(code: "{report_code}"){{                        
                                                    table(fightIDs: [0], dataType: DamageDone, hostilityType: Friendlies)
                                                    }}
                                                }}
                                            }}""" 
    return query

def make_healing_query(report_code):
    query = f"""query PlayerDungeonMetrics{{
                                            reportData{{
                                                report(code: "{report_code}"){{                        
                                                    table(fightIDs: [0], dataType: Healing, hostilityType: Friendlies)
                                                    }}
                                                }}
                                            }}""" 
    return query

def get_damage_and_healing(token, report_code):
    """
    Uses a token and a reportcode for warcraftlogs to get data for damage and healing

    Args:
        token (str): The access token to use for authorization.
        report_code (str): The reportcode for a report on warcraftlogs.

    Returns:
        DataFrame with the data.
    """
    #Damage part
    damage_query = make_query(token, make_damage_query(report_code))
    logger.info("dmg_query successful.")
    df_dmg_temp = pd.json_normalize(damage_query, record_path=['data', 'reportData', 'report', 'table', 'data', 'entries'])
    dmg_columns = ['name', 'type', 'itemLevel', 'total']
    df_damage = df_dmg_temp[dmg_columns]
    df_damage.columns = ['name', 'class', 'ilvl', 'Dps']
            
    #healing part
    healing_query = make_query(token, make_healing_query(report_code))
    logger.info("heal_query successful.")
    df_heal_temp = pd.json_normalize(healing_query, record_path=['data', 'reportData', 'report', 'table', 'data', 'entries'])
    heal_columns = ['name', 'total']
    df_heal = df_heal_temp[heal_columns]
    df_heal.columns = ['name', 'Healing']
            
    #Merge them
    merged_df = pd.merge(df_heal, df_damage, on='name')
    #Reorder columns
    healing_column = merged_df.pop('Healing')
    merged_df.insert(4, 'Healing', healing_column)

    return(merged_df)


In [113]:
#Functions for adding a timestamp for the report. 
def make_date_query(report_code):
    query = f"""query PlayerDungeonMetrics{{
                                            reportData{{
                                                report(code: "{report_code}"){{                        
                                                    startTime
                                                    }}
                                                }}
                                            }}""" 
    return query

def get_date(token, report_code):
    date_query = make_query(token, make_date_query(report_code))
    logger.info("date_query successful.")
    start_time = (date_query['data']['reportData']['report']['startTime'] / 1000)
    date_time = datetime.datetime.fromtimestamp(start_time)
    return date_time
    


In [148]:
# Utility functions

def print_nested_dict_keys(d, indentation=''):
    """
    Recursively prints all keys of a nested dictionary.
    
    Args:
        d (dict): The dictionary to inspect.
        indentation (str): A string to use for indentation to visualize the nesting.
    """
    for key, value in d.items():
        print(f"{indentation}{key}")
        
        # Check if the value is a dictionary and not empty
        if isinstance(value, dict) and value:
            # If it's a nested dictionary, call the function again (recursion)
            # with an increased indentation.
            print_nested_dict_keys(value, indentation + '  ')
        # Handle lists of dictionaries
        elif isinstance(value, list):
            for item in value:
                if isinstance(item, dict):
                    print_nested_dict_keys(item, indentation + '  ')

In [144]:
def main():
    logger.info("Starting the script")
    
    token = read_token()
    if not token:
        print("No token found, fetching new one...")
        client_id = os.getenv('CLIENT_ID')
        client_secret = os.getenv('CLIENT_SECRET')
        response = get_new_token(client_id, client_secret)
        token = response.json().get("WARCRAFTLOGS_TOKEN")
    
    logger.info("Autherization complete")

    if token:
        list_of_codes =["qTZ8J9rbgvj3KyAQ"]
        for code in list_of_codes:
            df_dmg_healing = get_damage_and_healing(token, code)
            start_time = pd.Series(get_date(token, code), name='date')
            start_time = start_time.repeat(5)
            gameID = get_gameID(token, code)
            gameID['date'] = start_time.values
            df = pd.merge(gameID, df_dmg_healing, on='name')
            print(df)

        

        

In [145]:
if __name__ == "__main__":
    main()

2025-09-04 11:14:48,414 - __main__ -INFO -Starting the script
2025-09-04 11:14:48,417 - __main__ -INFO -Autherization complete
2025-09-04 11:14:49,401 - __main__ -INFO -dmg_query successful.
2025-09-04 11:14:50,377 - __main__ -INFO -heal_query successful.
2025-09-04 11:14:51,348 - __main__ -INFO -date_query successful.
2025-09-04 11:14:52,349 - __main__ -INFO -gameID_query successful.


        name     gameID                    date        class  ilvl  \
0   Draeilan  128260472 2025-08-27 19:45:24.100      Paladin   699   
1  Angerfizt  109011842 2025-08-27 19:45:24.100       Hunter   705   
2   Kastorru  159797365 2025-08-27 19:45:24.100  DeathKnight   701   
3     Cragee  187346310 2025-08-27 19:45:24.100  DemonHunter   701   
4     Wendey  211686012 2025-08-27 19:45:24.100        Druid   705   

           Dps     Healing  
0   5155206024  1272504995  
1   9298203026   257291373  
2   8973656349   163314979  
3  10220036163   332729829  
4     43257842  2007266009  
